In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np

In [ ]:
# Lista de nombres de archivo de imágenes en el directorio
nombres_imagenes = os.listdir('./data')

In [ ]:
# Función para aplicar la transformada de Fourier a una imagen
def aplicar_fft_a_imagen(imagen, filtro_gaussiano=True, sigma=1.0):
    # Aplicar la transformada de Fourier 2D
    imagen_fft = np.fft.fft2(imagen)

    # Desplazar el componente de baja frecuencia al centro
    imagen_fft_desplazada = np.fft.fftshift(imagen_fft)

    # Calcular el logaritmo de la magnitud para una mejor visualización
    magnitud_fft = np.log(np.abs(imagen_fft_desplazada) + 1)

    # Aplicar un filtro Gaussiano para eliminar ruido
    if filtro_gaussiano:
        # Crear un filtro Gaussiano en el dominio de la frecuencia
        rows, cols = imagen.shape
        x = np.linspace(-cols/2, cols/2, cols)
        y = np.linspace(-rows/2, rows/2, rows)
        x, y = np.meshgrid(x, y)
        filtro_gaussiano = np.exp(-(x**2 + y**2) / (2 * sigma**2))

        # Aplicar el filtro Gaussiano a la imagen en el dominio de la frecuencia
        imagen_fft_filtrada = imagen_fft_desplazada * filtro_gaussiano

        # Deshacer el desplazamiento
        imagen_fft_filtrada = np.fft.ifftshift(imagen_fft_filtrada)

        # Aplicar la transformada inversa de Fourier
        imagen_filtrada = np.fft.ifft2(imagen_fft_filtrada).real

        return magnitud_fft, imagen_filtrada
    else:
        return magnitud_fft

# Función para cargar y las imagenes preprocesadas
def cargar_y_preprocesar_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Cargar imagen en escala de grises
    imagen = cv2.resize(imagen, (224, 224))  # Cambiar el tamaño de la imagen según las necesidades de tu modelo
    imagen = tf.keras.applications.mobilenet_v2.preprocess_input(imagen)  # Preprocesar la imagen para MobileNetV2
    return imagen

In [ ]:
# Procesar cada imagen en el directorio
for nombre_imagen in nombres_imagenes:
    ruta_imagen = os.path.join(directorio, nombre_imagen)

    # Cargar la imagen utilizando OpenCV
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Convierte la imagen a escala de grises

    # Aplicar la transformada de Fourier a la imagen
    imagen_fft = aplicar_fft_a_imagen(imagen)

    # Guardar la imagen procesada en un nuevo archivo
    nombre_imagen_salida = f'fft_{nombre_imagen}'
    ruta_imagen_salida = os.path.join(directorio, nombre_imagen_salida)
    cv2.imwrite(ruta_imagen_salida, imagen_fft)

print("Procesamiento de imágenes completado.")


In [ ]:
nombres_imagenes = [nombre for nombre in os.listdir(directorio) if nombre.startswith('fft_')]
modelo_neuronal = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)

# Crear listas para almacenar las imágenes y etiquetas
imagenes = []
etiquetas = []

# Cargar las imágenes procesadas y asignar etiquetas
for nombre_imagen in nombres_imagenes:
    ruta_imagen = os.path.join(directorio, nombre_imagen)
    imagen = cargar_y_preprocesar_imagen(ruta_imagen)
    etiqueta = 0  # Asigna la etiqueta adecuada a cada imagen
    imagenes.append(imagen)
    etiquetas.append(etiqueta)

# Convertir las listas a matrices NumPy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)

# Introduciriamos aqui el array de imagenes al modelo
resultados = modelo_neuronal.predict(imagenes)
